In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir('..')

In [ ]:
import polars as pl
from dotenv import load_dotenv
from pypi_llm.config import Config
from pypi_llm.description_cleaner import DescriptionCleaner, CLEANING_FAILED

load_dotenv()
config = Config()

In [ ]:
df = pl.read_csv("data/pypi_dataset.csv")
df = df.rename({
    "project_name" : "name",
    "project_description" : "description",
    "latest_project_version" : "latest_version",
    "project_number_of_downloads" : "number_of_downloads",
})
df = df.unique(subset="name")
df = df.filter(~pl.col("description").is_null())
df = df.sort("number_of_downloads", descending=True)

In [ ]:
df = DescriptionCleaner().clean(df, "description", "description_cleaned")
df = df.filter(~pl.col("description_cleaned").is_null())
df = df.filter(pl.col("description_cleaned")!=CLEANING_FAILED)

In [ ]:
from pypi_llm.vector_database_upserter import VectorDatabaseUpserter
upserter = VectorDatabaseUpserter(
    pinecone_token = config.PINECONE_TOKEN, 
    pinecone_index_name = config.PINECONE_INDEX_NAME,
    embeddings_model_name = config.EMBEDDINGS_MODEL
)

In [ ]:
upserter.upsert_polars(df, key_column="name", text_column="description_cleaned")

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(config.EMBEDDINGS_MODEL)
embeddings = model.encode(query)

In [ ]:
from pinecone import Pinecone, Index
pc = Pinecone(api_key=config.PINECONE_TOKEN)
index = pc.Index(config.PINECONE_INDEX_NAME)

index.query(
    namespace="ns1",
    vector=embeddings.tolist(),
    top_k=50,
    include_values=False
)